In [1]:
import pandas as pd
import numpy as np

## Descripción de los datos

In [2]:
df = pd.read_csv('datos.csv')

print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'datos.csv'

## Análisis de Correlación

In [ ]:
# Seleccionar solo las columnas numéricas para la correlación
df_numeric = df.select_dtypes(include=[np.number])

print(f"\nColumnas numéricas encontradas: {len(df_numeric.columns)}")
print("Columnas numéricas:", df_numeric.columns.tolist())

# Crear matriz de correlación
correlation_matrix = df_numeric.corr()

# Guardar la matriz de correlación en un archivo CSV
correlation_matrix.to_csv('matriz_correlacion.csv')
print("\nMatriz de correlación guardada en 'matriz_correlacion.csv'")

### Variables correlacionadas (con valor de correlación de [0.7, 1] y [-0.7, -1])

In [ ]:
# Crear lista de pares de correlaciones (excluyendo la diagonal y duplicados)
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):  # Solo j < i (parte inferior)
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        corr_value = correlation_matrix.iloc[i, j]
        if not np.isnan(corr_value) and abs(corr_value) >= 0.7:
            corr_pairs.append({
                'Variable1': col1,
                'Variable2': col2,
                'Correlación': corr_value,
                'Abs_Correlación': abs(corr_value)
            })

# Convertir a DataFrame y ordenar por valor absoluto
corr_df = pd.DataFrame(corr_pairs)
corr_df_sorted = corr_df.sort_values('Abs_Correlación', ascending=False)

print("\nCorrelaciones más fuertes:")
print(corr_df_sorted.to_string(index=False))

# Guardar análisis de correlaciones
corr_df_sorted.to_csv('analisis_correlaciones.csv', index=False)
print("\nAnálisis completo de correlaciones guardado en 'analisis_correlaciones.csv'")

variables = []

for dictionary in corr_pairs:
    variables.append(dictionary['Variable1'])
    variables.append(dictionary['Variable2'])
    
variables = set(variables)
print(variables)

## Análisis sobre reducción de dimensionalidad y ruido

Para reducir la dimensionalidad del espacio y el posible ruido utilizaremos dos técnicas: 

- Análisis de Componentes Principales (PCA) 

- Análisis de Componentes Independientes (ICA)

Para verificar cuál fue más efectiva haremos una comparación de cómo se comportó el algoritmo de aprendizae de máquinas seleccionado con ambas reducciones en un estudio posterior.

### Análisis de Componentes Principales (PCA)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")

Estrategia: eliminar variables con muchos missing values y imputar el resto

Eliminar columnas con más del 50% de valores faltantes

In [ ]:
threshold = len(df_numeric) * 0.5
df_clean = df_numeric.dropna(axis=1, thresh=threshold)

print(f"\nDespués de eliminar variables con >50% missing: {df_clean.shape}")

Imputar valores faltantes restantes con la mediana

In [ ]:
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df_clean), 
                         columns=df_clean.columns)

print(f"Después de imputación: {df_imputed.shape}")

#### Estandarización de los datos

In [ ]:
print("\nEstandarizando datos...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_imputed)

#### Aplicar PCA

In [ ]:
print("\nAplicando PCA...")
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

#### Análisis de varianza explicada

In [ ]:
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

Visualizar varianza explicada

In [ ]:
plt.figure(figsize=(15, 5))

Varianza explicada por componente

In [ ]:
plt.subplot(1, 3, 1)
plt.plot(range(1, len(explained_variance) + 1), explained_variance, 'bo-')
plt.xlabel('Componente Principal')
plt.ylabel('Varianza Explicada')
plt.title('Varianza por Componente')
plt.grid(True, alpha=0.3)

Varianza acumulada

In [ ]:
plt.subplot(1, 3, 2)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'ro-')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Acumulada')
plt.title('Varianza Acumulada')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% varianza')
plt.axhline(y=0.90, color='y', linestyle='--', label='90% varianza')
plt.axhline(y=0.85, color='r', linestyle='--', label='85% varianza')
plt.legend()
plt.grid(True, alpha=0.3)

Scree plot

In [ ]:
plt.subplot(1, 3, 3)
plt.bar(range(1, len(explained_variance) + 1), explained_variance)
plt.plot(range(1, len(explained_variance) + 1), explained_variance, 'ro-')
plt.xlabel('Componente Principal')
plt.ylabel('Varianza Explicada')
plt.title('Scree Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Encontrar el número óptimo de componentes

In [ ]:
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_85 = np.argmax(cumulative_variance >= 0.85) + 1

print(f"\nNúmero de componentes para:")
print(f"95% de varianza: {n_components_95}")
print(f"90% de varianza: {n_components_90}")
print(f"85% de varianza: {n_components_85}")

#### Aplicar PCA con número óptimo de componentes

In [ ]:
n_components_optimo = n_components_90
print(f"\nAplicando PCA con {n_components_optimo} componentes...")

pca_optimo = PCA(n_components=n_components_optimo)
X_pca_optimo = pca_optimo.fit_transform(X_scaled)

print(f"Dimensionalidad reducida: {X_pca_optimo.shape}")
print(f"Varianza total explicada: {pca_optimo.explained_variance_ratio_.sum():.4f}")

#### Análisis de componentes - coeficientes de las componentes

In [ ]:
loadings = pd.DataFrame(
    pca_optimo.components_.T,
    columns=[f'PC{i+1}' for i in range(n_components_optimo)],
    index=df_imputed.columns
)

print("\nVariables más importantes en cada componente:")
for i in range(min(3, n_components_optimo)):
    print(f"\nComponente PC{i+1}:")
    top_vars = loadings[f'PC{i+1}'].abs().sort_values(ascending=False).head(10)
    print(top_vars)

#### Visualización de los dos primeros componentes

In [ ]:
plt.figure(figsize=(15, 5))

Gráfico de loadings para los primeros dos componentes

In [ ]:
plt.subplot(1, 2, 1)
plt.scatter(loadings['PC1'], loadings['PC2'], alpha=0.7)

Añadir etiquetas para las variables más importantes

In [ ]:
for idx, row in loadings.iterrows():
    if abs(row['PC1']) > 0.2 or abs(row['PC2']) > 0.2:
        plt.annotate(idx, (row['PC1'], row['PC2']), fontsize=8)

plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.7)
plt.xlabel(f'PC1 ({explained_variance[0]:.2%} varianza)')
plt.ylabel(f'PC2 ({explained_variance[1]:.2%} varianza)')
plt.title('Loadings - PC1 vs PC2')
plt.grid(True, alpha=0.3)

Gráfico de proyección de datos en los primeros dos componentes

In [ ]:
plt.subplot(1, 2, 2)
plt.scatter(X_pca_optimo[:, 0], X_pca_optimo[:, 1], alpha=0.6, s=10)
plt.xlabel(f'PC1 ({explained_variance[0]:.2%} varianza)')
plt.ylabel(f'PC2 ({explained_variance[1]:.2%} varianza)')
plt.title('Proyección de Datos - PC1 vs PC2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Crear el dataset reducido

In [ ]:
df_pca = pd.DataFrame(
    X_pca_optimo,
    columns=[f'PC{i+1}' for i in range(n_components_optimo)]
)

print(f"\nDataset reducido creado: {df_pca.shape}")

#### Análisis de contribución de variables

In [ ]:
variable_importance = pd.DataFrame({
    'variable': df_imputed.columns,
    'importance': np.sum(loadings.abs().values, axis=1)
}).sort_values('importance', ascending=False)

print("\nTop 20 variables más importantes en el PCA:")
print(variable_importance.head(20))

In [ ]:
# Guardar dataset reducido
df_pca.to_csv('dataset_reducido_pca.csv', index=False)

# Guardar información del PCA
pca_info = pd.DataFrame({
    'component': [f'PC{i+1}' for i in range(n_components_optimo)],
    'explained_variance': pca_optimo.explained_variance_ratio_,
    'cumulative_variance': np.cumsum(pca_optimo.explained_variance_ratio_)
})
pca_info.to_csv('informacion_pca.csv', index=False)

# Guardar loadings
loadings.to_csv('loadings_pca.csv')

print("\n✅ Análisis PCA completado!")
print(f"✅ Dimensionalidad reducida de {df_numeric.shape[1]} a {n_components_optimo} variables")
print(f"✅ Varianza explicada: {pca_optimo.explained_variance_ratio_.sum():.2%}")
print(f"✅ Archivos guardados:")
print("   - dataset_reducido_pca.csv")
print("   - informacion_pca.csv")
print("   - loadings_pca.csv")

### Análisis de Componentes Independientes (ICA)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy import stats
from kneed import KneeLocator
from statsmodels.tsa.stattools import acf
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")

Se vuelven a cargar los datos para trabajar nuevamente con el original

In [ ]:
print("Cargando y preprocesando datos...")
df = pd.read_csv('datos.csv')

df_numeric = df.select_dtypes(include=[np.number])

print(f"Dataset original: {df.shape}")
print(f"Variables numéricas: {df_numeric.shape}")


In [ ]:
print("\nManejando valores faltantes...")
print(f"Valores faltantes por variable:\n{df_numeric.isnull().sum().sort_values(ascending=False).head(10)}")

threshold = len(df_numeric) * 0.5
df_clean = df_numeric.dropna(axis=1, thresh=threshold)

print(f"\nDespués de eliminar variables con >50% missing: {df_clean.shape}")

imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df_clean), 
                         columns=df_clean.columns)

print(f"Después de imputación: {df_imputed.shape}")

print("\nEstandarizando datos...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_imputed)

### Determinar el número óptimo de componentes para ICA

In [ ]:
def find_optimal_ica_components(X, max_components=50):
    """
    Encuentra el número óptimo de componentes para ICA basado en la curtosis promedio
    """
    kurtosis_values = []
    component_range = range(2, min(max_components, X.shape[1]) + 1)
    
    for n_components in component_range:
        ica = FastICA(n_components=n_components, random_state=42, max_iter=1000)
        try:
            S = ica.fit_transform(X)
            kurt_vals = stats.kurtosis(S, axis=0)
            avg_kurtosis = np.mean(np.abs(kurt_vals))
            kurtosis_values.append(avg_kurtosis)
        except:
            kurtosis_values.append(0)
    
    return component_range, kurtosis_values

print("Buscando número óptimo de componentes...")
component_range, kurtosis_values = find_optimal_ica_components(X_scaled)

plt.figure(figsize=(15, 5))

In [ ]:
plt.subplot(1, 3, 1)
plt.plot(component_range, kurtosis_values, 'bo-')
plt.xlabel('Número de Componentes')
plt.ylabel('Curtosis Promedio (abs)')
plt.title('Curtosis vs Número de Componentes ICA')
plt.grid(True, alpha=0.3)

In [ ]:
knee_finder = KneeLocator(component_range, kurtosis_values, curve='concave', direction='increasing')
n_components_optimo = knee_finder.knee

if n_components_optimo is None:
    n_components_optimo = min(20, X_scaled.shape[1] // 3)  # Valor por defecto

plt.axvline(x=n_components_optimo, color='red', linestyle='--', 
           label=f'Óptimo: {n_components_optimo} componentes')
plt.legend()

#### Aplicar ICA con el número óptimo de componentes

In [ ]:
print(f"\nAplicando ICA con {n_components_optimo} componentes...")
ica = FastICA(n_components=n_components_optimo, random_state=42, max_iter=2000)
S = ica.fit_transform(X_scaled)  # Componentes independientes
A = ica.mixing_  # Matriz de mezcla

print(f"Forma de componentes independientes: {S.shape}")
print(f"Forma de matriz de mezcla: {A.shape}")

#### Análisis de los componentes independientes

In [ ]:
component_stats = []
for i in range(S.shape[1]):
    component = S[:, i]
    component_stats.append({
        'Componente': f'IC{i+1}',
        'Media': np.mean(component),
        'Desviación': np.std(component),
        'Curtosis': stats.kurtosis(component),
        'Skewness': stats.skew(component)
    })

df_component_stats = pd.DataFrame(component_stats)
print("\nEstadísticas de los componentes independientes:")
print(df_component_stats)

Visualizar distribución de los componentes

In [ ]:
plt.subplot(1, 3, 2)
plt.bar(range(1, len(component_stats) + 1), [abs(cs['Curtosis']) for cs in component_stats])
plt.xlabel('Componente Independiente')
plt.ylabel('|Curtosis|')
plt.title('No-Gaussianidad de Componentes ICA')
plt.grid(True, alpha=0.3)

#### Análisis de la matriz de mezcla

In [ ]:
loadings_ica = pd.DataFrame(
    A,
    columns=[f'IC{i+1}' for i in range(n_components_optimo)],
    index=df_imputed.columns
)

print("\nVariables más importantes en cada componente independiente:")
for i in range(min(3, n_components_optimo)):
    print(f"\nComponente IC{i+1}:")
    top_vars = loadings_ica[f'IC{i+1}'].abs().sort_values(ascending=False).head(10)
    print(top_vars)

#### Visualización de componentes

In [ ]:
plt.subplot(1, 3, 3)

In [ ]:
important_vars = loadings_ica.abs().max(axis=1).sort_values(ascending=False).head(20).index
loadings_subset = loadings_ica.loc[important_vars]

sns.heatmap(loadings_subset, cmap='RdBu_r', center=0, annot=True, fmt='.2f')
plt.title('Matriz de Mezcla ICA (Top 20 variables)')
plt.tight_layout()
plt.show()

#### VIsualización de distribuciones de componentes

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i in range(min(6, n_components_optimo)):
    ax = axes[i]
    component = S[:, i]
    
    # Histograma y KDE
    ax.hist(component, bins=50, density=True, alpha=0.7, color='skyblue')
    
    # Añadir distribución normal para comparación
    xmin, xmax = ax.get_xlim()
    x = np.linspace(xmin, xmax, 100)
    p = stats.norm.pdf(x, np.mean(component), np.std(component))
    ax.plot(x, p, 'k', linewidth=2, label='Distribución Normal')
    
    ax.set_title(f'Componente IC{i+1}\nCurtosis: {stats.kurtosis(component):.2f}')
    ax.set_xlabel('Valor')
    ax.set_ylabel('Densidad')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Análisis de contribución de variables

In [ ]:
variable_importance_ica = pd.DataFrame({
    'variable': df_imputed.columns,
    'importance': np.sum(loadings_ica.abs().values, axis=1)
}).sort_values('importance', ascending=False)

print("\nTop 20 variables más importantes en el ICA:")
print(variable_importance_ica.head(20))

#### Crear dataset reducido

In [ ]:
df_ica = pd.DataFrame(
    S,
    columns=[f'IC{i+1}' for i in range(n_components_optimo)]
)

print(f"\nDataset transformado con ICA: {df_ica.shape}")

#### Análisis de separación de fuentes

In [ ]:
if n_components_optimo >= 2:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Scatter plots de pares de componentes
    axes[0, 0].scatter(S[:, 0], S[:, 1], alpha=0.6, s=10)
    axes[0, 0].set_xlabel('IC1')
    axes[0, 0].set_ylabel('IC2')
    axes[0, 0].set_title('IC1 vs IC2')
    axes[0, 0].grid(True, alpha=0.3)
    
    if n_components_optimo >= 3:
        axes[0, 1].scatter(S[:, 0], S[:, 2], alpha=0.6, s=10)
        axes[0, 1].set_xlabel('IC1')
        axes[0, 1].set_ylabel('IC3')
        axes[0, 1].set_title('IC1 vs IC3')
        axes[0, 1].grid(True, alpha=0.3)
    
    if n_components_optimo >= 4:
        axes[1, 0].scatter(S[:, 1], S[:, 2], alpha=0.6, s=10)
        axes[1, 0].set_xlabel('IC2')
        axes[1, 0].set_ylabel('IC3')
        axes[1, 0].set_title('IC2 vs IC3')
        axes[1, 0].grid(True, alpha=0.3)
    
    if n_components_optimo >= 1:
        autocorr = acf(S[:, 0], nlags=20)
        axes[1, 1].stem(range(len(autocorr)), autocorr)
        axes[1, 1].set_xlabel('Lag')
        axes[1, 1].set_ylabel('Autocorrelación')
        axes[1, 1].set_title('Autocorrelación de IC1')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Guardar dataset transformado
df_ica.to_csv('dataset_ica_transformado.csv', index=False)

# Guardar matriz de mezcla
loadings_ica.to_csv('matriz_mezcla_ica.csv')

# Guardar información de componentes
component_info = pd.DataFrame({
    'component': [f'IC{i+1}' for i in range(n_components_optimo)],
    'kurtosis': [stats.kurtosis(S[:, i]) for i in range(n_components_optimo)],
    'skewness': [stats.skew(S[:, i]) for i in range(n_components_optimo)],
    'std': [np.std(S[:, i]) for i in range(n_components_optimo)]
})
kurtosis_ica = np.mean(np.abs(stats.kurtosis(S, axis=0)))
component_info.to_csv('informacion_componentes_ica.csv', index=False)

print("\n✅ Análisis ICA completado!")
print(f"✅ Dimensionalidad reducida de {df_numeric.shape[1]} a {n_components_optimo} componentes independientes")
print(f"✅ Curtosis promedio: {kurtosis_ica:.4f}")
print(f"✅ Archivos guardados:")
print("   - dataset_ica_transformado.csv")
print("   - matriz_mezcla_ica.csv")
print("   - informacion_componentes_ica.csv")